In [1]:
# import libraries

from binance.client import Client
import pandas as pd
from datetime import datetime

# Your api_key is your public key, i.e. other people can send you coins through this.
api_key ='EYLz9h7MwAVxmA54GfF8xgEOnds4AuER3jLRbGEhzuQHuyLlMw7YEmMKR9hGPMUf'
# Your api_secret should never be shared with anyone, and is required to access your coins
api_secret = ''
client = Client(api_key, api_secret)
print("logged in")




logged in


In [2]:
#Binance's export is alway in .xlsx format
raw_trade_1 = pd.read_excel('Export Recent Trade History.xlsx')
#If more than one export is taken, they can be grouped together and de-duplicated using the Date as the parameter
raw_trade_2 = pd.read_excel('Export Recent Trade History 2.xlsx')

#Combine all raw export history
raw_combined = pd.concat([raw_trade_2,raw_trade_1])

#Checkpoint copy
export_history = raw_combined.copy()

#Change formatting of Date to give UK date format
export_history['Date(UTC)'] =export_history['Date(UTC)'].apply(lambda x:
                                                      pd.Timestamp(x).strftime('%d-%m-%Y %H:%M:%S'))

#Drop all duplicates in case of exports having overlapping timeframes
export_history=export_history.drop_duplicates('Date(UTC)').reset_index(drop=True)
export_history


,Date(UTC),Market,Type,Price,Amount,Total,Fee,Fee Coin
0,28-06-2021 11:30:00,ETHGBP,BUY,1000.00,0.700000,700.000000,0,ETH
1,21-06-2021 12:32:26,ETHGBP,BUY,1000.00,0.200000,200.000000,0,ETH
2,22-06-2021 20:32:26,BTCGBP,BUY,40000.00,0.500000,20000.000000,0,BTC
3,28-02-2021 15:32:26,ETHGBP,BUY,950.00,0.308270,292.856500,0,ETH
4,28-02-2021 15:15:30,BTCGBP,BUY,31940.00,0.000548,17.503120,0,BTC
5,28-02-2021 15:15:27,BTCGBP,BUY,31940.00,0.002582,82.469080,0,BTC
6,20-02-2021 11:14:36,BTCGBP,BUY,40064.00,0.004902,196.393728,0,BTC
7,08-02-2021 16:17:12,ETHGBP,BUY,1288.50,0.114320,147.301320,0,ETH
8,08-02-2021 11:04:36,ETHGBP,BUY,1212.14,0.000990,1.200019,0,ETH
9,08-02-2021 11:04:35,ETHGBP,BUY,1212.14,0.080000,96.971200,0,ETH


In [13]:
#Create master dataframe, using only unique types of coin imported

data = {'Coin': sorted(export_history["Fee Coin"].unique())}
df = pd.DataFrame (data)

#Create range of unique coins
unique_range=range(len(export_history["Fee Coin"].unique()))

#Sums all quantities of coin filtering the type of coin and adds it to an empty list
quantity=[]
for i in unique_range:
    quantity.append(export_history.groupby('Fee Coin').sum().loc[sorted(export_history["Fee Coin"].unique())[i]]['Amount'])
df['Quantity']= quantity

#Sums all purchases of coin filtering the type of coin and adds it to an empty list
purchase=[]
for i in unique_range:
    purchase.append(export_history.groupby('Fee Coin').sum().loc[sorted(export_history["Fee Coin"].unique())[i]]['Total'])
df['Money Spent'] = purchase

#Uses API to call a list of all exchange rates
prices = client.get_all_tickers()
live_prices=[]
for p in prices:
    for i in unique_range:
        if p['symbol'] =='{}GBP'.format(sorted(export_history["Fee Coin"].unique())[i]):
            live_prices.append(float(p['price']))

#Now master dataframe can be expanded by adding columns created    
df['Live price per coin'] = live_prices
df['Current Value']=df["Live price per coin"]*df['Quantity']
#Profit/Loss is calculated by current value of the coin minus initial investment
df['P/L']=df['Current Value'] - df['Money Spent']

#Checkpoint
df_pre = df.copy()
#Sum all columns
total_cols = df.sum(axis=0)
df_pre = df.append(total_cols, ignore_index=True)

#Profit/Loss expressed as a percentage
df_pre['P/L %']= (df_pre['Current Value'] - df_pre['Money Spent'])*100/df_pre['Money Spent']

#Checkpoint
df_reorder = df_pre.copy()
#df_reorder = df_reorder.applymap('{:.2f}%'.format)
df_reorder
#Reorder columns
df_reorder=df_reorder[['Coin', 'Current Value',
       'P/L', 'P/L %',  'Money Spent', 'Quantity','Live price per coin' ]]

#len(unique_range) is used as index to give the bottom position of the dataframe
#Rename the 1st column, bottom positon as 'Total' instead of all the coin names being concatenated together
df_reorder.iloc[len(unique_range), df_reorder.columns.get_loc('Coin')] = 'Total'
#Adding all coin currencies together doesn't make sense due to different units, hence N/A is added
df_reorder.iloc[len(unique_range), df_reorder.columns.get_loc('Quantity')] = 'N/A'
#Same logic applies for Live price per coin
df_reorder.iloc[len(unique_range), df_reorder.columns.get_loc('Live price per coin')] = 'N/A' 

#Checkpoint
df_totals = df_reorder.copy()
#Sort final dataframe to give Total row at the top and reset index
df_tracker=df_totals.sort_values(by=['P/L'],ascending=False).reset_index(drop=True)
df_tracker




,Coin,Current Value,P/L,P/L %,Money Spent,Quantity,Live price per coin
0,Total,23477.571147,1742.876181,8.018867,21734.694967,N/A,N/A
1,BTC,21374.166159,1077.800231,5.310311,20296.365928,0.508032,42072.5
2,ETH,2103.404988,665.075949,46.239486,1438.329039,1.40358,1498.6
